In [ ]:
import datetime
import sys

import numpy
import scipy.stats
%matplotlib widget
from matplotlib import pyplot
from matplotlib.dates import num2date, date2num
import netCDF4
from ipywidgets import widgets

sys.path.append('../extern/fabm-mizer/python')
import mizer

## Starting from a constant plankton size spectrum

In [ ]:
# More about the parameters below can be found in:
# * the source code: https://github.com/pmlmodelling/fabm-mizer/blob/master/src/size_structured_population.F90
# * the mizer vignette: https://mran.microsoft.com/snapshot/2017-02-04/web/packages/mizer/vignettes/mizer_vignette.pdf
# * Supplementary information of Blanchard et al. 2014: https://doi.org/10.1111/1365-2664.12238

def run(nday=36500, c_ini=1., verbose=True, **parameter_overrides):
    parameters = dict(
        # spectrum partition
        w_min=1e-3,                   # minimum size for the predator spectrum (g)
        w_inf=1e6,                    # maximum size for the predator spectrum (g)
        nclass=100,                   # number of size classes for the predator spectrum (g)

        # temperature dependence
        T_dependence=1,               # temperature dependence of rates (0=none, 1=Arrhenius)
        T_ref=13.,                    # reference temperature at which all rates must be given (degrees Celsius)
        E_a=0.63,                     # activation energy for Arrhenius relationship (eV)

        # predator-prey preference
        beta=100,                     # optimal predator : prey wet mass ratio (-)
        sigma=float(numpy.log(10.)),  # standard devation of predator-prey preference (ln g)

        # clearance, ingestion, growth efficiency
        gamma=156,                    # scale factor for clearance rate (m3/yr/g^q) = actual rate for individuals of 1 g
        q=0.82,                       # allometric exponent for clearance rate
        h=1e9,                        # scale factor for maximum ingestion rate (g/yr/g^n)
        #n=2./3.,                     # allometric exponent for maximum ingestion rate
        alpha=0.2,                    # gross growth efficiency or assimilation efficiency (-)
        ks=0.,                        # standard metabolism (1/yr/g^p)

        # mortality
        z0_type=1,                    # type of intrinsic mortality (0=constant, 1=allometric function)
        z0pre=0.1,                    # scale factor for intrinsic mortality (1/yr/g^z0exp)
        z0exp=-0.25,                  # allometric exponent for intrinsic mortality
        z_spre=0.2,                   # scale factor for senescence mortality (1/yr)
        w_s=1000.,                    # reference ("starting") individual wet mass for senescence mortality (g)
        z_s=0.3,                      # allometric exponent for senescence mortality

        # recruitment
        SRR=0,                        # stock-recruitment relationship (0=constant recruitment, 1=equal to reproductive output, 2=Beverton-Holt)
        recruitment=0.,               # constant recruitment rate for smallest size class (#/yr)

        # fishing
        fishing_type=1,               # fishing type (0=none, 1: knife-edge, 2: logistic/sigmoid, 3: linearly increasing)
        w_minF=1.25,                  # minimum individual wet mass for fisheries mortality
        F=0.4                         # maximum fisheries mortality (knife-edge: constant mortality for individuals > w_minF)
    )

    # Apply any parameter overrides that the user supplied as named arguments
    parameters.update(parameter_overrides)

    # Set up constant prey spectrum
    prey_collection = mizer.PreyCollection(mizer.Prey('plankton', [1e-14, 1e-3], c_ini))
    prey_collection = mizer.GriddedPreyCollection(prey_collection, extend=False)

    # Create mizer model (note prescribed ambient temperature, depth of water column)
    m = mizer.Mizer(prey=prey_collection, parameters=parameters, temperature=13., depth=50., recruitment_from_prey=2, verbose=False)

    # Time integrate for requested number of days and return result
    start = datetime.datetime(2000, 1, 1)
    stop = start + datetime.timedelta(days=nday)
    time = numpy.arange(date2num(start), date2num(stop))
    return m.run(time, spinup=0, verbose=verbose, save_spinup=False, save_loss_rates=True, save_f=True, dt=1.)

def run_and_plot(ax, label=None, *args, **kwargs):
    result = run(verbose=False, *args, **kwargs)
    result.plot_spectrum(ax=ax, style='-', label=label)

In [ ]:
result = run(nday=1000, c_ini=1.)

In [ ]:
def start_stop(play=True):
    if play:
        anim.event_source.start()
    else:
        anim.event_source.stop()
anim = result.animate_spectrum(repeat=False)
widgets.interact(start_stop);

In [ ]:
fig, ax = pyplot.subplots()

In [ ]:
run_and_plot(ax, label='default')

In [ ]:
run_and_plot(ax, F=0, label='no fishing')

In [ ]:
run_and_plot(ax, z0pre=0., z_spre=0., label='no residual mortality')